# Phase Estimation of Quantum Walks

In [ ]:
#  Copyright 2023 Google LLC
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      https://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.

## Heisenberg limited phase estimation
Implements Heisenberg-Limited Phase Estimation of the Qubitized Quantum Walks as described in Section-II B. of [Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity](https://arxiv.org/abs/1805.03662)

In [ ]:
import cirq
import numpy as np
import attrs

from qualtran import GateWithRegisters, Signature, Register, QFxp 
from qualtran._infra.gate_with_registers import get_named_qubits
from qualtran.bloqs.qubitization_walk_operator import QubitizationWalkOperator
from qualtran.bloqs.qubitization_walk_operator_test import get_walk_operator_for_1d_Ising_model
from qualtran.bloqs.hubbard_model import get_walk_operator_for_hubbard_model
from qualtran.cirq_interop.testing import GateHelper
from qualtran.cirq_interop.t_complexity_protocol import t_complexity
from qualtran.drawing.flame_graph import show_flame_graph
from qualtran.resource_counting.generalizers import cirq_to_bloqs

In [ ]:
def get_resource_state(m: int):
    r"""Returns a state vector representing the resource state on m qubits from Eq.17 of Ref-1.
    
    Returns a numpy array of size 2^{m} representing the state vector corresponding to the state
    $$
        \sqrt{\frac{2}{2^m + 1}} \sum_{n=0}^{2^{m}-1} \sin{\frac{\pi(n + 1)}{2^{m}+1}}\ket{n}
    $$
    
    Args:
        m: Number of qubits to prepare the resource state on.
    
    Ref:
        1) [Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity]
            (https://arxiv.org/abs/1805.03662)
            Eq. 17
    """
    den = 1 + 2 ** m
    norm = np.sqrt(2 / den)
    return norm * np.sin(np.pi * (1 + np.arange(2**m)) / den)        


@attrs.frozen
class ResourceStatePhaseEstimation(GateWithRegisters):
    """Heisenberg limited phase estimation circuit for learning eigenphase of `walk`.
    
    The method yields an OPTREE to construct Heisenberg limited phase estimation circuit 
    for learning eigenphases of the `walk` operator with `m` bits of accuracy. The 
    circuit is implemented as given in Fig.2 of Ref-1.
    
    Args:
        walk: Qubitization walk operator.
        m: Number of bits of accuracy for phase estimation. 
        
    Ref:
        1) [Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity]
            (https://arxiv.org/abs/1805.03662)
            Fig. 2
    """
    walk: QubitizationWalkOperator
    m: int

    @property
    def signature(self) -> Signature:
        return Signature([*self.walk.signature, Register('phase_reg', QFxp(self.m, self.m))])

    def pretty_name(self) -> str:
        return f'ResourceStatePhaseEstimation[walk][{self.m}]'

    def decompose_from_registers(self, **quregs):
        m, walk = self.m, self.walk
        reflect = walk.reflect
        walk_regs = {reg.name: quregs[reg.name] for reg in self.walk.signature}
        reflect_regs = {reg.name: walk_regs[reg.name] for reg in reflect.signature}
        
        reflect_controlled = reflect.controlled(control_values=[0])
        walk_controlled = walk.controlled(control_values=[1])
    
        m_qubits = quregs['phase_reg']
    
        yield walk_controlled.on_registers(**walk_regs, control=m_qubits[0])
        for i in range(1, m):
            yield reflect_controlled.on_registers(control=m_qubits[i], **reflect_regs)
            yield walk.on_registers(**walk_regs)
            walk = walk ** 2
            yield reflect_controlled.on_registers(control=m_qubits[i], **reflect_regs)
            

def phase_estimation(walk, m_bits):
    pe_walk = ResourceStatePhaseEstimation(walk, m_bits)
    gh = GateHelper(pe_walk)
    state_prep = cirq.StatePreparationChannel(get_resource_state(m_bits), name='chi_m')
    yield state_prep.on(*gh.quregs['phase_reg'])
    yield from cirq.decompose_once(gh.operation)
    yield cirq.qft(*gh.quregs['phase_reg'], inverse=True)

In [ ]:
x_dim, y_dim = 20, 20
t = 20
mu = 4 * t
N = x_dim * y_dim * 2
qlambda = 2 * N * t + (N * mu) // 2
delta_E = t / 100
m_bits = int(np.log2(qlambda * np.pi * np.sqrt(2) / delta_E))
walk = get_walk_operator_for_hubbard_model(x_dim, y_dim, t, mu)
print(walk.t_complexity())
show_flame_graph(walk, file_path='out.svg')

In [ ]:
num_sites: int = 6
eps: float = 1e-2
m_bits: int = 4


circuit = cirq.Circuit(phase_estimation(get_walk_operator_for_1d_Ising_model(num_sites, eps), m_bits))
print(circuit)
print(t_complexity(circuit[1:-1]))
pe_walk = ResourceStatePhaseEstimation(get_walk_operator_for_1d_Ising_model(num_sites, eps), m_bits)
show_flame_graph(pe_walk)

## Resource estimates for 1D Ising model using generic SELECT / PREPARE 

In [ ]:
num_sites: int = 200
eps: float = 1e-5
m_bits: int = 14

walk = get_walk_operator_for_1d_Ising_model(num_sites, eps)

circuit = cirq.Circuit(phase_estimation(walk, m_bits))
%time result = t_complexity(circuit[1:-1])
print(result)

## Resource estimates for 2D Hubbard model using specialized SELECT / PREPARE 
Phase estimation of walk operator for 2D Hubbard Model using SELECT and PREPARE circuits from Section V of https://arxiv.org/abs/1805.03662

In [ ]:
x_dim, y_dim = 20, 20
t = 20
mu = 4 * t
N = x_dim * y_dim * 2
qlambda = 2 * N * t + (N * mu) // 2
delta_E = t / 100
m_bits = int(np.log2(qlambda * np.pi * np.sqrt(2) / delta_E))
walk = get_walk_operator_for_hubbard_model(x_dim, y_dim, t, mu)
circuit = cirq.Circuit(phase_estimation(walk, m_bits))
%time result = t_complexity(circuit[1:-1])
print(result)

## Resource estimates for THC model using specialized SELECT / PREPARE 

In [ ]:
from qualtran.bloqs.chemistry.thc.select_bloq import SelectTHC
from qualtran.bloqs.chemistry.thc.prepare import PrepareTHC
num_spat = 4
num_mu = 8
t_l = np.random.normal(0, 1, size=num_spat)
zeta = np.random.normal(0, 1, size=(num_mu, num_mu))
zeta = 0.5 * (zeta + zeta.T)
thc_prep = PrepareTHC.from_hamiltonian_coeffs(t_l, zeta, num_bits_state_prep=8)

num_mu = 8
num_spin_orb = 2 * 4
thc_sel = SelectTHC(num_mu=num_mu, num_spin_orb=num_spin_orb, num_bits_theta=12)

walk = QubitizationWalkOperator(select=thc_sel, prepare=thc_prep)
m_bits = 4
pe_walk = ResourceStatePhaseEstimation(walk, m_bits)

circuit = cirq.Circuit(phase_estimation(walk, m_bits))
%time result = t_complexity(circuit[1:-1])
print(result)

show_flame_graph(pe_walk)